# Imports.

In [17]:
import pandas as pd
import numpy as np
import scipy

from comp_econ import comp_econ as ce


# Problem 1

## Define A and b per the Miranda and Fackler textbook.

In [18]:
A = np.matrix([
    [54, 14, -11, 2],
    [14, 50, -4, 29],
    [-11, -4, 55, 22],
    [2, 29, 22, 95]
])
print("Matrix A:")
print(A)

b = np.matrix(np.ones(shape=(A.shape[0], 1)))
print("\nVector b:")
print(b)



Matrix A:
[[ 54  14 -11   2]
 [ 14  50  -4  29]
 [-11  -4  55  22]
 [  2  29  22  95]]

Vector b:
[[1.]
 [1.]
 [1.]
 [1.]]


## Solve for x using LU Decomposition.

In [19]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu.html

print("Running LU Decomposition method.")
P, L, U = scipy.linalg.lu(A)
assert np.allclose(A, (P @ L @ U)), "Check LU decomposition."

# Ax = b
# A = LU, or very close to A.
# --> LUx = b
# --> x = (LU)^-1 * b
LU_inv = scipy.linalg.inv((L @ U))
x_lu = (LU_inv @ b).round(4)
print("Vector x_lu:")
print(x_lu)

Running LU Decomposition method.
Vector x_lu:
[[ 0.0189]
 [ 0.0168]
 [ 0.0234]
 [-0.0004]]


## Solve for x using both Gauss-Jacobi and Gauss-Seidel methods.

In [20]:
print("Running Gauss-Jacobi method.")
x_gj, num_iter_gj = ce.gjacobi(A, b, np.zeros(shape=(4, 1)))
x_gj = x_gj.round(4)
print("Vector x_gj:")
print(x_gj)

print("\nRunning Gauss-Seidel method.")
x_gs, num_iter_gs = ce.gseidel(A, b, np.zeros(shape=(4, 1)))
x_gs = x_gs.round(4)
print("Vector x_gs:")
print(x_gs)

Running Gauss-Jacobi method.
Gauss-Jacobi method converged in 18 iterations.
Vector x_gj:
[[ 0.0189]
 [ 0.0168]
 [ 0.0234]
 [-0.0004]]

Running Gauss-Seidel method.
Gauss-Seidel method converged in 9 iterations.
Vector x_gs:
[[ 0.0189]
 [ 0.0168]
 [ 0.0234]
 [-0.0004]]


## Check the answers of all 3 methods.

In [21]:
assert np.allclose(x_lu, x_gj, x_gs), "Solution vectors should all be approximately equal."

# Problem 2.